In [ ]:
pip install numpy==1.26.4


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
board = [" " for _ in range(9)]

def check_winner(b, player):
    winningconditions = [(0,1,2),(3,4,5),(6,7,8),
                      (0,3,6),(1,4,7),(2,5,8),
                      (0,4,8),(2,4,6)]
    return any(b[i] == b[j] == b[k] == player for i, j, k in winningconditions)
def ismatchdraw(b):
    return all(cell != " " for cell in b)

In [ ]:

def minimax(b, depth, alpha, beta, is_maximizing):
    if check_winner(b, "0"):
        return 1
    if check_winner(b, "X"):
        return -1
    if ismatchdraw(b):
        return 0

    if is_maximizing:
        maximumeval = float('-inf')
        for i in range(9):
            if b[i] == " ":
                b[i] = "0"
                eval = minimax(b, depth + 1, alpha, beta, False)
                b[i] = " "
                maximumeval = max(maximumeval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
        return maximumeval
    else:
        minimumeval = float('inf')
        for i in range(9):
            if b[i] == " ":
                b[i] = "X"
                eval = minimax(b, depth + 1, alpha, beta, True)
                b[i] = " "
                minimumeval = min(minimumeval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
        return minimumeval

#move for ai
def best_move(b):
    best_value = float('-inf')
    move = -1
    for i in range(9):
        if b[i] == " ":
            b[i] = "0"
            moving_value = minimax(b, 0, float('-inf'), float('inf'), False)
            b[i] = " "
            if moving_value > best_value:
                best_value = moving_value
                move = i
    return move



In [ ]:
def play(cell):
    global board
    if board[cell] != " ":
        return board.copy(), "**Move is invalid. Try a different cell!**", *[gr.update(value=board[i], interactive=(board[i] == " ")) for i in range(9)]
    
    #player move
    board[cell] = "X"
    if check_winner(board, "X"):
        temp = board.copy()
        board = [" " for _ in range(9)]
        return temp, "Congrats!! You won the match!!", *[gr.update(value=temp[i], interactive=False) for i in range(9)]
    
    if ismatchdraw(board):
        temp = board.copy()
        board = [" " for _ in range(9)]
        return temp, "Match draw!!!", *[gr.update(value=temp[i], interactive=False) for i in range(9)]
    
#AI move
    ai = best_move(board)
    if ai != -1:
        board[ai] = "0"
    
    if check_winner(board, "0"):
        temp = board.copy()
        board = [" " for _ in range(9)]
        return temp, "AI wins the match!!", *[gr.update(value=temp[i], interactive=False) for i in range(9)]
    
    if ismatchdraw(board):
        temp = board.copy()
        board = [" " for _ in range(9)]
        return temp, "Match Draw!!", *[gr.update(value=temp[i], interactive=False) for i in range(9)]
    
    return board.copy(), "Your move!", *[gr.update(value=board[i], interactive=(board[i] == " ")) for i in range(9)]

def reset_game():
    global board
    board = [" " for _ in range(9)]
    return board.copy(), "Game reset. Start your move!!", *[gr.update(value=" ", interactive=True) for _ in range(9)]

css = """
.cell-button {
  height: 80px;
  width: 80px;
  font-size: 32px;
  margin: 5px;
}
"""

with gr.Blocks(css=css) as mytictactoeapp:
    gr.Markdown("<h1 style='text-align: center;'>Tic-Tac-Toe AI</h1>")
    gr.Markdown("<h1 style='text-align: center;'>Play against an **Unbeatable AI** using Minimax + Alpha-Beta Pruning</h1>")

    state = gr.State(value=board.copy())
    status = gr.Textbox(value="Your move!", label="Status", interactive=False)
    
    button_list = []  
    with gr.Column():
        for row in range(3):
            with gr.Row():
                for col in range(3):
                    idx = row * 3 + col
                    btn = gr.Button(value=" ", elem_classes=["cell-button"])
                    button_list.append(btn)
    for i in range(9):
        button_list[i].click(
            fn=lambda i=i: play(i),
            inputs=[], 
            outputs=[state, status] + button_list
        )
    
    gr.Button("Reset Game").click(
        fn=reset_game,
        inputs=[],
        outputs=[state, status] + button_list
    )

mytictactoeapp.launch(share=True)
